# Training a convolutional neural net to predict Absolute Magnitude by DR14 APOGEE_Distances Value Added Catalogue using astroNN

*  **Henry W.H. Leung** - Astronomy Undergraduate, University of Toronto - henrysky
* Project advisor: **Jo Bovy** - Professor, Department of Astronomy and Astrophysics, University of Toronto - jobovy
*  Contact Henry: henrysky.leung [at] mail.utoronto.ca
* This tutorial is created on 10/Feb/2018 with Keras 2.1.3, Tensorflow 1.6.0, Nvidia CuDNN 7.0 for CUDA 9.0 (Optional), Python 3.6.3 Win10 x64
* Since astroNN is still in active development, this tutorial is creacted with the following astroNN 0.9.1 Alpha

* astroNN github: https://github.com/henrysky/astroNN

* astroNN documentation: http://astronn.readthedocs.io/

The data we will be using is BPG StarHorse median posterior A(V) (Bayesian; Santiago et al. 2016, Queiroz et al. 2017)

Stellar Parameters (Surface Effective Temperature, Surface Gravity and Iron) are from ASPCAP

### Data preparation

In [ ]:
from astroNN.datasets import H5Loader
from astroNN.datasets import load_apogee_distances
from astroNN.datasets import xmatch
import numpy as np

# Setup a H5loader instance
# __train.h5 is a compiled dataset described at 
# http://astronn.readthedocs.io/en/latest/compile_datasets.html#compiling-dataset
loader = H5Loader('__train.h5')

# We dont need error for CNN
loader.load_err = False

# x is sprectra and y is abundances
x, y = loader.load()

# By using load_entry() method, the resulting array always correspoinds to the x, y loaded from load()
ra = loader.load_entry('RA')
dec = loader.load_entry('DEC')
kmag = loader.load_entry('Kmag')

# load the apogee distances
ra_dist, dec_dist, absmag, absmag_err = load_apogee_distances(dr=14, metric='absmag', cuts=True)

# xmatch the two
idx_1, idx_2, sep = xmatch(ra, ra_dist, colRA1=ra, colDec1=dec, colRA2=ra_dist, colDec2=dec_dist, swap=False)

# finally our training data
x = x[idx_1]

# We just need the first 3, [teff, logg, Fe]
y = (y[:, 0:3])[idx_1]

# Combine ASPCAP labels with absolute magnitude
y_train = np.zeros((y.shape[0], 4))
y_train[:,0:3]=y
y_train[:,3]=absmag[idx_2]

# randomly select 2000 spectra as test set
test_idx = np.random.randint(0, y_train.shape[0], 2000)

# Testing Data
x_test = x[test_idx]
y_test = y_train[test_idx]

# Training data
x_train = np.delete(x, test_idx, axis=0)
y_train = np.delete(y_train, test_idx, axis=0)

# K mag too so we can convert absolute magnitude back to distance
kmag_test = kmag[test_idx]
kmag_train = np.delete(kmag, test_idx, axis=0)

### Train a neural network

In [ ]:
from astroNN.models import ApogeeCNN
cnn = ApogeeCNN()

# proporly set the tragetname
cnn.targetname = ['teff', 'logg', 'Fe', 'absmag']

# We only need a small neural net in this case
cnn.num_hidden = [128,64]

# setup the maximium epoxh the neural net can be trained
cnn.max_epochs = 30

# setup the initial learning rate
cnn.lr=0.0001

# setup the folder name
cnn.folder_name = 'bovy_meeting'

# train neural net here
cnn.train(x_train,y_train)

### The way you can load the folder back if you restarted python

In [ ]:
from astroNN.models import load_folder
cnn = load_folder('bovy_meeting')

### Use the trained neural network to predict on test spectra

In [ ]:
prediction = cnn.test(x_test)

### Use aspcap_residue_plot() method to plot residue

In [ ]:
from astroNN.gaia import absmag_to_pc

# convert our predicition of absolute magnitude to distance
prediction[:,3] = absmag_to_pc(prediction[:,3], kmag_test)

# convert our absolute magnitude ground truth to distance
y_test[:,3] = absmag_to_pc(y_test[:,3], kmag_test)

# prediction is the prediction from the trained neural network
# y_test is the ground truth for test set
cnn.aspcap_residue_plot(prediction, y_test)

## The result

## For absmag, the plot is actually showing distance (in parsec), the graphs are generated in an automated function, sorry for the confusion

### `m` stands for median
### `S hat` stands for normalized Median Absolute Derivation
### `S` stands for astropy's mad_std (Corrected Median Absolute Derivation)

![title](absmag_test.png)
![title](Fe_test.png)
![title](logg_test.png)
![title](teff_test.png)